In [ ]:
# Import packages needed
import gc
import pickle
from tkinter import Tk
from tkinter.filedialog import askopenfilename, asksaveasfilename
import platform


from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.widgets import RectangleSelector, PolygonSelector
import numpy as np
from skimage import io, draw
import mplcursors
import IPython.display as Disp
from ipywidgets import widgets
import cv2
from osgeo import gdal
import pandas as pd

import general_funcs

# OS related settings
if platform.system() == 'Windows':
    # %matplotlib nbagg
    # Sometimes tk/qt will not let cells rerun after an ERROR occurs
    # %matplotlib tk
    %matplotlib qt
elif platform.system() == 'Darwin':
    Tk().withdraw()
    %matplotlib osx
elif platform == 'linux' or platform == 'linux2':
    ;

# This line of "print" must exist right after %matplotlib command, 
# otherwise JN will hang on the first import statement after this.
print('Interactive plot activated')


# Extend width of Jupyter Notebook Cell to the size of browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Put two plot together
def onselect(vert):
    ;
    
def set_mtps(mtp, mtp_ref):
    def on_click(event):
        # Image ref
        if event.inaxes in [axes[0]]:
            mtp_ref.append([event.xdata, event.ydata])
            axes[0].text(event.xdata, event.ydata, 'MTP_ref ' + str(len(mtp_ref)))
            
            circle_ref = patches.Circle([event.xdata, event.ydata], radius=50, color='green')
            axes[0].add_patch(circle_ref)
        elif event.inaxes in [axes[1]]:
            mtp.append([event.xdata, event.ydata])
            axes[1].text(event.xdata, event.ydata, 'MTP ' + str(len(mtp)))
            
            circle = patches.Circle([event.xdata, event.ydata], radius=50, color='yellow')
            axes[1].add_patch(circle)
    return(on_click)


def reset_mtps(mtp, mtp_ref):
    def on_key(event):
        if event.key == 'escape' and event.inaxes in [axes[0]]:
            for t in axes[0].texts:
                t.set_visible(False)
            axes[0].texts.clear()
            axes[0].patches.clear()
            mtp_ref.clear()
        elif event.key == 'escape' and event.inaxes in [axes[1]]:
            for t in axes[1].texts:
                t.set_visible(False)
            axes[1].texts.clear()
            axes[1].patches.clear()
            mtp.clear()
    return(on_key)

image_file_ref = '/Users/lj/Library/Mobile Documents/com~apple~CloudDocs/Developer/RGB-IR_Field_Image_Processing/data/field_image/BRC/BRC_20190814_141033_Thermal.tif'
img_ref = io.imread(image_file_ref)
print("Image Ref Shape: ", img_ref.shape)
layer_IR_ref = img_ref[:, :, 0]
layer_mask_ref = img_ref[:, :, 1]
# Set background to black
layer_IR_ref[np.where(layer_mask_ref == 0)] = 0
vmin_ref, vmax_ref = general_funcs.cal_vmin_vmax(layer_IR_ref, layer_mask_ref)


image_file = '/Users/lj/Library/Mobile Documents/com~apple~CloudDocs/Developer/RGB-IR_Field_Image_Processing/data/field_image/BRC/BRC_20191008_145306_Thermal.tif'
img = io.imread(image_file)
print("Image Shape: ", img.shape)
layer_IR = img[:, :, 0]
layer_mask = img[:, :, 1]
# Set background to black
layer_IR[np.where(layer_mask == 0)] = 0
vmin, vmax = general_funcs.cal_vmin_vmax(layer_IR, layer_mask)

# Select MTPs from image ref
plt.close('all')
fig, axes = plt.subplots(2, 1, figsize=(7, 7))
plt.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95)

mtp_ref = []
myax_ref = axes[0].imshow(layer_IR_ref, cmap='gist_gray', vmin=vmin_ref, vmax=vmax_ref)
mtp = []
myax = axes[1].imshow(layer_IR, cmap='gray', vmin=vmin, vmax=vmax)

ps_ref = PolygonSelector(axes[0], onselect, useblit=True, lineprops=dict(linewidth=0), )
ps = PolygonSelector(axes[1], onselect, useblit=True, lineprops=dict(linewidth=0), )

cid_set_mtps = fig.canvas.mpl_connect('button_press_event', set_mtps(mtp, mtp_ref))
cid_reset_mtps = fig.canvas.mpl_connect('key_press_event', reset_mtps(mtp, mtp_ref))
print(cid_set_mtps)

In [ ]:
# Show selected point on previous plot
fig.canvas.mpl_disconnect(cid_set_mtps)
fig.canvas.mpl_disconnect(cid_reset_mtps)
ps_ref.disconnect_events()
ps.disconnect_events()


ds_ref = gdal.Open(image_file_ref)
gt_ref = ds_ref.GetGeoTransform()
mtp_ref_geo = []

for point in mtp_ref:
    geo_loc = general_funcs.pix2geo(point, gt_ref)
    mtp_ref_geo.append(geo_loc)
    

ds = gdal.Open(image_file)
gt = ds.GetGeoTransform()
mtp_geo = []
for point in mtp:
    geo_loc = general_funcs.pix2geo(point, gt)
    mtp_geo.append(geo_loc)
    
mtp_ref_geo = np.array(mtp_ref_geo)
mtp_geo = np.array(mtp_geo)

shift_geo = mtp_geo - mtp_ref_geo


ps = PolygonSelector(axes[1], onselect, useblit=True, lineprops=dict(linewidth=0), )

selected_pos = []

def mouse_click(event):
    if event.inaxes in [axes[1]]:
        pix_loc = np.round([event.xdata, event.ydata])
        print(pix_loc)
        selected_pos.append(pix_loc)
        
        geo_loc = general_funcs.pix2geo(pix_loc, gt)
        point_shift_geo = general_funcs.find_point_in_img_ref(pix_loc, mtp, mtp_ref, shift_geo)
        geo_loc_ref = geo_loc - point_shift_geo
        pix_loc_ref = general_funcs.geo2pix(geo_loc_ref, gt_ref)
        pix_loc_ref_before_transform = general_funcs.geo2pix(geo_loc, gt_ref)
        
        
        
        circle1 = patches.Circle(pix_loc_ref, radius=50)
        circle2 = patches.Circle(pix_loc_ref_before_transform, radius=50, color='red')
        axes[0].add_patch(circle1)
        axes[0].add_patch(circle2)
    
    
print(fig.canvas)
fig.canvas.mpl_connect('button_press_event', mouse_click)